### Simple self-attention mechanism

In [5]:
import torch
text = "Your journey starts with one step"
# assume the embedding layer is initialized with the following values
inputs = torch.tensor(
    [
        [0.43, 0.15, 0.89],
        [0.55, 0.87, 0.66],
        [0.57, 0.85, 0.64],
        [0.22, 0.58, 0.33],
        [0.77, 0.25, 0.10],
        [0.05, 0.80, 0.55]
    ]
)

In [12]:
query = inputs[1]

In [13]:
query

tensor([0.5500, 0.8700, 0.6600])

In [8]:
attn_scores_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(inputs[1], x_i)

In [9]:
attn_scores_2

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])

In [10]:
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)

In [11]:
attn_weights_2

tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])

In [ ]:
# %%timeit
# context_vector_2 = torch.zeros((6000, 3))
# attn_weights_2_2 = torch.cat([attn_weights_2] * 1000)
# for i, x_i in enumerate(torch.cat([inputs] * 1000)):
#     context_vector_2 += attn_weights_2_2[i] * x_i

113 ms ± 1.78 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [21]:
context_vector_2

tensor([0.4419, 0.6515, 0.5683])

In [ ]:
# %%timeit
# attn_weights_2_2 = torch.cat([attn_weights_2] * 1000)
# context_vector_2 = attn_weights_2_2 @ torch.cat([inputs] * 1000)

200 μs ± 568 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [34]:
context_vector_2 = attn_weights_2 @ inputs

In [35]:
context_vector_2

tensor([0.4419, 0.6515, 0.5683])

In [37]:
attn_scores = inputs @ inputs.T

In [38]:
attn_scores

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])

In [44]:
import numpy as np
assert np.allclose(attn_scores[1], attn_scores_2)

In [45]:
attn_weights = torch.softmax(attn_scores, dim=-1)

In [46]:
attn_weights

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])

In [48]:
assert np.allclose(attn_weights_2, attn_weights[1])

In [51]:
assert np.allclose(torch.sum(attn_weights, dim=-1), 1)

In [52]:
context_vectors = attn_weights @ inputs

In [53]:
context_vectors

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])

In [54]:
assert np.allclose(context_vectors[1], context_vector_2)